In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [2]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_df = pd.read_csv(commit_guru_path)
#     print(commit_guru_df.columns)
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','author_date','fileschanged',
                                                   'author_date_unix_timestamp', 'commit_message','commit_hash',
                                                  'classification', 'fix','fixes',],axis=1)

    df = commit_guru_df
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    y.fillna(False,inplace=True)
    df['Bugs'] = y
    df.dropna(inplace=True)
    cols = df.columns.tolist()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

# def load_data(project):
#     understand_path = 'data/understand_files_all/' + project + '_understand.csv'
#     commit_guru_path = 'data/commit_guru/' + project + '.csv'
#     understand_df = pd.read_csv(understand_path)
#     understand_df = understand_df.dropna(axis = 1,how='all')
#     cols_list = understand_df.columns.values.tolist()
#     for item in ['Kind', 'Name','commit_hash', 'Bugs']:
#         if item in cols_list:
#             cols_list.remove(item)
#             cols_list.insert(0,item)
#     understand_df = understand_df[cols_list]
#     commit_guru_df = pd.read_csv(commit_guru_path)
#     cols = understand_df.columns.tolist()
    
#     commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
#                                                    'author_email','fileschanged','author_date',
#                                                    'author_date_unix_timestamp', 'commit_message',
#                                                   'classification', 'fix', 'contains_bug','fixes',],axis=1)
#     commit_cols = commit_guru_df.columns.values.tolist()
#     understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
#     df = understand_df.merge(commit_guru_df,on='commit_hash')
#     cols = df.columns.tolist()
#     cols = cols[1:] + [cols[0]]
#     df = df[cols]
#     for item in ['Kind', 'Name','commit_hash']:
#         if item in cols:
#             df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     commit_cols.remove('commit_hash')
#     commit_cols.append('Bugs')
#     df = df[commit_cols]
#     y = df.Bugs
#     X = df.drop('Bugs',axis = 1)
#     cols = X.columns
#     scaler = MinMaxScaler()
#     X = scaler.fit_transform(X)
#     X = pd.DataFrame(X,columns = cols)
#     return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [3]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

In [93]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

In [4]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [5]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/With_median_commit_guru_data/with_diff_models/process_1000_nb.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      1.00      0.99        38
        True       0.00      0.00      0.00         1

    accuracy                           0.97        39
   macro avg       0.49      0.50      0.49        39
weighted avg       0.95      0.97      0.96        39

+++++++++++++++++   qpython  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   friendlychat-android  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.92      1.00      0.96        11
        True       0.00      0.00      0.00         1

    accuracy                           0.92        12
   macro avg       0.46      0.50      0.48        12
weighted avg       0.84      0.92      0.88        12

+++++++++++++++++   paho.mqtt.android  +++++++++++++++++
              precision    recall  f1-score   support

       False       

              precision    recall  f1-score   support

       False       0.97      1.00      0.98        64
        True       0.00      0.00      0.00         2

    accuracy                           0.97        66
   macro avg       0.48      0.50      0.49        66
weighted avg       0.94      0.97      0.95        66

+++++++++++++++++   FastScroll  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.88      1.00      0.94        15
        True       0.00      0.00      0.00         2

    accuracy                           0.88        17
   macro avg       0.44      0.50      0.47        17
weighted avg       0.78      0.88      0.83        17

+++++++++++++++++   recyclerviewhelper  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   MaterialFavoriteButton  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.94      0.97        17
        True       

              precision    recall  f1-score   support

       False       0.89      0.89      0.89        18
        True       0.00      0.00      0.00         2

    accuracy                           0.80        20
   macro avg       0.44      0.44      0.44        20
weighted avg       0.80      0.80      0.80        20

+++++++++++++++++   Hacktoberfest-Data-Structure-and-Algorithms  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.99      0.96      0.98       183
        True       0.12      0.50      0.20         2

    accuracy                           0.96       185
   macro avg       0.56      0.73      0.59       185
weighted avg       0.98      0.96      0.97       185

+++++++++++++++++   ToggleButton  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.72      0.84        18
        True       0.00      0.00      0.00         0

    accuracy                           0.72    

              precision    recall  f1-score   support

       False       1.00      0.90      0.95       242
        True       0.04      1.00      0.08         1

    accuracy                           0.90       243
   macro avg       0.52      0.95      0.51       243
weighted avg       1.00      0.90      0.94       243

+++++++++++++++++   alicloud-android-demo  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      0.88      0.92        65
        True       0.11      0.33      0.17         3

    accuracy                           0.85        68
   macro avg       0.54      0.61      0.54        68
weighted avg       0.93      0.85      0.89        68

+++++++++++++++++   hugo  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.64      0.78        36
        True       0.13      1.00      0.24         2

    accuracy                           0.66        38
   macro avg       0.57

              precision    recall  f1-score   support

       False       0.97      0.62      0.76        55
        True       0.09      0.67      0.15         3

    accuracy                           0.62        58
   macro avg       0.53      0.64      0.45        58
weighted avg       0.93      0.62      0.72        58

+++++++++++++++++   GroupedRecyclerViewAdapter  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.88      0.71      0.79        21
        True       0.14      0.33      0.20         3

    accuracy                           0.67        24
   macro avg       0.51      0.52      0.49        24
weighted avg       0.79      0.67      0.72        24

+++++++++++++++++   PowerMenu  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   TagsEditText  +++++++++++++++++
              precision    recall  f1-score   support

       False       1.00      0.87      0.93        15
        True       0.3

...

              precision    recall  f1-score   support

       False       0.88      0.94      0.91       687
        True       0.37      0.21      0.26       112

    accuracy                           0.84       799
   macro avg       0.63      0.57      0.59       799
weighted avg       0.81      0.84      0.82       799

+++++++++++++++++   handlebars.java  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.99      0.38      0.55       236
        True       0.39      0.99      0.56        94

    accuracy                           0.55       330
   macro avg       0.69      0.69      0.55       330
weighted avg       0.82      0.55      0.55       330

+++++++++++++++++   stringtemplate4  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.78      0.93      0.85       239
        True       0.74      0.44      0.55       109

    accuracy                           0.78       348
   macro avg      

              precision    recall  f1-score   support

       False       0.89      0.90      0.89       695
        True       0.36      0.35      0.35       118

    accuracy                           0.82       813
   macro avg       0.63      0.62      0.62       813
weighted avg       0.81      0.82      0.82       813

+++++++++++++++++   PageTurner  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.84      0.73      0.78       327
        True       0.41      0.58      0.48       108

    accuracy                           0.69       435
   macro avg       0.63      0.66      0.63       435
weighted avg       0.74      0.69      0.71       435

+++++++++++++++++   heroic  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.75      0.94      0.84       295
        True       0.44      0.13      0.20       105

    accuracy                           0.73       400
   macro avg       0.60      0.5

              precision    recall  f1-score   support

       False       0.92      0.20      0.33       486
        True       0.22      0.92      0.35       116

    accuracy                           0.34       602
   macro avg       0.57      0.56      0.34       602
weighted avg       0.78      0.34      0.33       602

+++++++++++++++++   pebble  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.73      0.96      0.83       277
        True       0.69      0.20      0.31       123

    accuracy                           0.73       400
   macro avg       0.71      0.58      0.57       400
weighted avg       0.72      0.73      0.67       400

+++++++++++++++++   CameraView  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.66      0.74      0.70        82
        True       0.83      0.76      0.80       136

    accuracy                           0.76       218
   macro avg       0.74      0.7

              precision    recall  f1-score   support

       False       0.75      0.90      0.82       327
        True       0.61      0.34      0.43       149

    accuracy                           0.72       476
   macro avg       0.68      0.62      0.63       476
weighted avg       0.71      0.72      0.70       476

+++++++++++++++++   opsu  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.89      0.18      0.30       182
        True       0.51      0.97      0.67       158

    accuracy                           0.55       340
   macro avg       0.70      0.58      0.48       340
weighted avg       0.71      0.55      0.47       340

+++++++++++++++++   metrics  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.83      0.91      0.86       714
        True       0.31      0.18      0.23       167

    accuracy                           0.77       881
   macro avg       0.57      0.54    

              precision    recall  f1-score   support

       False       0.79      0.96      0.87      1140
        True       0.53      0.15      0.24       346

    accuracy                           0.77      1486
   macro avg       0.66      0.56      0.55      1486
weighted avg       0.73      0.77      0.72      1486

+++++++++++++++++   nokogiri  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.79      0.93      0.85      1255
        True       0.44      0.18      0.25       385

    accuracy                           0.75      1640
   macro avg       0.61      0.55      0.55      1640
weighted avg       0.70      0.75      0.71      1640

+++++++++++++++++   rapidoid  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.83      0.89      0.86      1686
        True       0.24      0.15      0.19       374

    accuracy                           0.76      2060
   macro avg       0.53      0.5

In [11]:
for key in final_result.keys():
    print(len(final_result[key].values()))
    print(key,np.nanmedian(list(final_result[key].values())))

1001
precision 0.55
1001
recall 0.55
1001
pf 0.21
1001
f1 0.53
1001
g 0.53
1001
auc 0.6181818181818183
1001
pci_20 0.62
1001
ifa 1.0


In [ ]:
_precision = list(precision_list.values())
_recall = list(recall_list.values())
_pf = list(pf_list.values())
_f1 = list(f1_list.values())
_g = list(g_list.values())
_auc = list(auc_list.values())
print(np.median(_precision),np.median(_recall),np.median(_pf),np.median(_f1),np.median(_g),np.median(_auc))
fig = plt.figure(num=None, figsize = (20,4), facecolor='w', edgecolor='k')
ax = fig.add_subplot(161)
ax.boxplot(_precision)
ax.set_title('Precision',size = 15)
ax = fig.add_subplot(162)
ax.boxplot(_recall)
ax.set_title('Recall',size = 15)
ax = fig.add_subplot(163)
ax.boxplot(_pf)
ax.set_title('pf',size = 15)
ax = fig.add_subplot(164)
ax.boxplot(_f1)
ax.set_title('f1',size = 15)
ax = fig.add_subplot(165)
ax.boxplot(_g)
ax.set_title('g',size = 15)
ax = fig.add_subplot(166)
ax.boxplot(_auc)
ax.set_title('auc',size = 15)
# fig.savefig('without_process.png')